In [1]:
using CSV, DataFrames, Distances
cities = CSV.read("cities_p.csv");

In [2]:
dists = Dict{Tuple{Int,Int},Float64}()

Dict{Tuple{Int64,Int64},Float64} with 0 entries

In [3]:
function get_score(cities, subm_path)
    global dists
    all_ids = cities[:CityId]
    all_x = cities[:X]
    all_y = cities[:Y]

    incs = 0
    score = 0.0
    pimp = 0.0
#     println(subm_path[1:25])
    for i in 1:length(subm_path)-1
        c_idx = subm_path[i]+1
        n_idx = subm_path[i+1]+1
        if !haskey(dists, (c_idx,n_idx))
            p1 = [all_x[c_idx],all_y[c_idx]]
            p2 = [all_x[n_idx],all_y[n_idx]]
            stepSize = euclidean(p1,p2)
            dists[(c_idx,n_idx)] = stepSize
        else
            stepSize = dists[(c_idx,n_idx)]    
        end
        if (i % 10 == 0 && !cities[:primes][subm_path[i]+1])
            pimp += 0.1*stepSize
            stepSize *= 1.1
            incs += 1
        end
        # print(stepSize)
        score += stepSize
    end
    return score, incs, pimp
end

get_score (generic function with 1 method)

In [4]:
#0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20
#  1   2   3   4   5   6   7   8   9   10   11  12    13   14   15   16   17   18   19   20
function move_x_to_y(path, x, y)
    new_path = copy(path)
    if x > y
        at_y = new_path[y]
        new_path[y] = new_path[x]
        for p in x:-1:y+2
            new_path[p] = new_path[p-1] 
        end
        new_path[y+1] = at_y
    else
        at_y = new_path[y]
        new_path[y] = new_path[x]   
        for p in x:y-2
            new_path[p] = new_path[p+1] 
        end
        new_path[y-1] = at_y
    end
    return new_path
end

move_x_to_y (generic function with 1 method)

In [5]:
@assert [5,1,2,3,4] == move_x_to_y([1,2,3,4,5],5,1)
@assert [2,3,4,5,1] == move_x_to_y([1,2,3,4,5],1,5)

In [6]:
function improve(cities, subm_path)
    all_ids = cities[:CityId]
    all_x = cities[:X]
    all_y = cities[:Y]

    incs = 0
    score = 0.0
    pimp = 0.0
    mods = zeros(Int,10)
    last = 0
    improved = true
    random_impros = 0
    while improved 
        score = 0.0
        improved = false
        for i in 1:length(subm_path)-1
            c_idx = subm_path[i]+1
            n_idx = subm_path[i+1]+1
            p1 = [all_x[c_idx],all_y[c_idx]]
            p2 = [all_x[n_idx],all_y[n_idx]]
            stepSize = euclidean(p1,p2)

            if i > last && cities[:primes][subm_path[i]+1]
                mod = i % 10
                if mod != 0 && i > 20 && i < length(subm_path)-20
                    rst = i-mod-9
                    rnd = rst+31
                    old_score,old_incs,_ = get_score(cities, subm_path[rst:rnd]) 
                    
                    new_subm_path = move_x_to_y(subm_path,i,i-mod)
                    new_score,new_incs,_ = get_score(cities, new_subm_path[rst:rnd])
                    if new_score < old_score
                        println("From: ", rst, " to ",rnd)
                        subm_path[rst:rnd] = new_subm_path[rst:rnd]
                        improved = true
                        last = i
                        println("Improved...")
                        println("old_incs: ", old_incs)
                        println("new_incs: ", new_incs)
                        break
                    else
                        new_subm_path = move_x_to_y(subm_path,i,i+10-mod)
                        new_score,new_incs,_ = get_score(cities, new_subm_path[rst:rnd])
                        if new_score < old_score
                            println("From: ", rst, " to ",rnd)
                            subm_path[rst:rnd] = new_subm_path[rst:rnd]
                            improved = true
                            last = i
                            println("Improved...")
                            println("old_incs: ", old_incs)
                            println("new_incs: ", new_incs)
                            break
                        end
                    end
                end
                mods[(i % 10) + 1] += 1
            end
            # print(stepSize)
            if i % 10 == 0 && !cities[:primes][subm_path[i]+1]
                pimp += 0.1*stepSize
                stepSize *= 1.1
                incs += 1
            end

            score += stepSize
        end
    end
#     println("mod: ", mods)
    return subm_path, score, incs, pimp
end

improve (generic function with 1 method)

In [9]:
subm_df = CSV.read("new_submission_1.csv");
subm_path = subm_df[:Path];
@time score, incs, pimp = get_score(cities, subm_path)
println("Score: ", score)
println("Incs: ", incs)
println("Possible Improvement: ", pimp)
println("Distance to 1: ", score-1516337.34)
println("Distance to 50: ", score-1533208.44)

  0.175634 seconds (2.67 M allocations: 79.222 MiB, 6.80% gc time)
Score: 1.5295388770946239e6
Incs: 17954
Possible Improvement: 13706.812628186466
Distance to 1: 13201.537094623782
Distance to 50: -3669.5629053760786


In [7]:
cluster = Dict{Tuple{Int,Int},Vector{Int}}()
for i=1:size(cities)[1]
    x = convert(Int,ceil(cities[:X][i]))
    y = convert(Int,ceil(cities[:Y][i]))
    x1 = x - (x % 15) + 15
    y1 = y - (y % 15) + 15
    if !haskey(cluster, (x1,y1))
        cluster[(x1,y1)] =  []
    end
    push!(cluster[(x1,y1)], i)
end

In [8]:

function improve_k()
    primes = findall(cities[:primes] .== true);
    fname = "new_submission_99000.csv"
    for k in 1:10000
        if k % 10 == 0
            println("k: ", k)
        end
        subm_df = CSV.read(fname);

        subm_path = collect(skipmissing(subm_df[:Path]));

        c1 = rand(2:size(cities)[1]-1)
        r1 = findfirst(subm_path .== c1-1)
        
        x = convert(Int,ceil(cities[:X][c1]))
        y = convert(Int,ceil(cities[:Y][c1]))
        x1 = x - (x % 15) + 15
        y1 = y - (y % 15) + 15
        clustervals = cluster[(x1,y1)]
        @time score, incs, pimp = get_score(cities, subm_path)
        for val in clustervals
            if val == c1
                continue
            end
            r2 = findfirst(subm_path .== val-1)
#             println("r1: ", r1)
#             println("r2: ", r2)
            p1 = [cities[:X][subm_path[r1]+1],cities[:Y][subm_path[r1]+1]]
            p2 = [cities[:X][subm_path[r2]+1],cities[:Y][subm_path[r2]+1]]
#             println("Dist: ", euclidean(p1,p2))
            new_subm_path = move_x_to_y(subm_path,r1,r2)
            new_subm_path = move_x_to_y(new_subm_path,r1,r2)
            nscore, incs, pimp = get_score(cities, new_subm_path)
            if score-nscore > 0
                fname = "new_submission_1.csv"
                println("Score before: ", score)
                println("Score after:  ", nscore)
                df = DataFrame(Path=new_subm_path)
                CSV.write("new_submission_1.csv", df)
                println("Score impro: ", score-nscore)
                println("Incs: ", incs)
                println("Possible Improvement: ", pimp)
                println("Distance to 1: ", nscore-1516337.34)
                println("Distance to 50: ", nscore-1533208.44)
                break
            end
        end
    end
end
improve_k()

  0.475360 seconds (3.16 M allocations: 116.625 MiB, 21.31% gc time)
  0.071302 seconds (863.41 k allocations: 19.210 MiB, 4.01% gc time)
  0.070166 seconds (863.41 k allocations: 19.210 MiB, 8.22% gc time)
  0.062934 seconds (863.41 k allocations: 19.210 MiB, 6.56% gc time)
  0.060390 seconds (863.41 k allocations: 19.210 MiB)
  0.066053 seconds (863.41 k allocations: 19.210 MiB, 4.26% gc time)
  0.066719 seconds (863.41 k allocations: 19.210 MiB, 4.40% gc time)
  0.110499 seconds (863.41 k allocations: 19.210 MiB, 27.29% gc time)
  0.075657 seconds (863.41 k allocations: 19.210 MiB, 3.37% gc time)
k: 10
  0.084371 seconds (863.41 k allocations: 19.210 MiB, 6.71% gc time)
  0.061365 seconds (863.41 k allocations: 19.210 MiB, 4.16% gc time)
  0.062992 seconds (863.41 k allocations: 19.210 MiB)
  0.072690 seconds (863.41 k allocations: 19.210 MiB, 5.94% gc time)
  0.062283 seconds (863.41 k allocations: 19.210 MiB, 5.93% gc time)
  0.072440 seconds (863.41 k allocations: 19.210 MiB, 8.6

InterruptException: InterruptException:

In [51]:
subm_df = CSV.read("new_submission_58000.csv");

subm_path = collect(skipmissing(subm_df[:Path]));
new_subm_path, score, _, _ = improve(cities, subm_path)

From: 2581 to 2612
Improved...
old_incs: 2
new_incs: 1
From: 22141 to 22172
Improved...
old_incs: 2
new_incs: 1
From: 23781 to 23812
Improved...
old_incs: 3
new_incs: 2
From: 57621 to 57652
Improved...
old_incs: 3
new_incs: 2
From: 59251 to 59282
Improved...
old_incs: 3
new_incs: 2
From: 62651 to 62682
Improved...
old_incs: 3
new_incs: 2
From: 63021 to 63052
Improved...
old_incs: 3
new_incs: 2
From: 81501 to 81532
Improved...
old_incs: 2
new_incs: 1
From: 96791 to 96822
Improved...
old_incs: 3
new_incs: 2
From: 113941 to 113972
Improved...
old_incs: 2
new_incs: 1
From: 114251 to 114282
Improved...
old_incs: 3
new_incs: 2
From: 115601 to 115632
Improved...
old_incs: 2
new_incs: 1
From: 118051 to 118082
Improved...
old_incs: 3
new_incs: 2
From: 135081 to 135112
Improved...
old_incs: 3
new_incs: 2
From: 135891 to 135922
Improved...
old_incs: 3
new_incs: 2
From: 141221 to 141252
Improved...
old_incs: 2
new_incs: 1
From: 143501 to 143532
Improved...
old_incs: 3
new_incs: 2
From: 162251 to 1

([0, 47239, 177242, 167366, 38941, 22121, 134856, 182992, 139226, 70519  …  195889, 20433, 78596, 71430, 153911, 75405, 40230, 48816, 78934, 0], 1.5243819332457557e6, 163846, 121689.55814023325)